# 🧠 LLM Price Estimator Project – Notes
## 📌 Overview
This project **(without using fine-tuning)** evaluates a **Large Language Model (LLM)** (specifically Meta's **LLaMA 3.1-8B**) to **predict prices from text** descriptions. The model is tested against a dataset where each entry contains text with embedded price info (e.g., product ads or listings). The model's price predictions are then evaluated for accuracy.  The next steps after this project are to fine-tune the model.

---
## 📦 Key Libraries
- transformers, datasets – Hugging Face tools for models and datasets
- bitsandbytes – For 4-bit/8-bit quantization (efficient model inference)
- peft – Parameter-Efficient Fine-Tuning
- trl – For RL-based model tuning (used indirectly)
- matplotlib, tqdm, re, math – Utilities for plotting and evaluation

## 🧠 Models Used
 - LLAMA_3_1 = "meta-llama/Meta-Llama-3.1-8B"
 - LLAMA_3_2 = "meta-llama/Meta-Llama-3.2-1B"
 - QWEN_2_5 = "Qwen/Qwen2.5-7B"
 - GEMMA_2 = "google/gemma-2-9b"
 - PHI_3   = "microsoft/Phi-3-medium-4k-instruct"

## Selected base model:
 - BASE_MODEL = LLAMA_3_1

## 📊 Dataset
Dataset is pulled from Hugging Face Hub:
- dataset = load_dataset(DATASET_NAME)
- train = dataset['train']
- test = dataset['test']
- Each entry contains:
    - text: a descriptive string containing a price
    - price: the ground truth numeric value

## 🧪 Tokenizer Exploration
- Helper function to inspect how models tokenize numbers:
```
def investigate_tokenizer(model_name)
 """Shows token IDs for numbers like 0, 1, 10, 100, etc."""
```

- Useful for understanding how models perceive numerics.
## 🤖 Model Loading with Quantization
- Uses 4-bit quantization to reduce memory usage and enable fast inference:
```
quant_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16,
  bnb_4bit_quant_type="nf4"
)
```

## Tokenizer and model loading:
```
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=quant_config)
```
## 🔍 Price Extraction
Function to extract price from text:
```
def extract_price(s):
  """Pulls a float value after "Price is $"""
```

## 📈 Inference Function
Runs text through the model and extracts predicted price:
```
def model_predict(prompt):
    """Encodes prompt, generates output, extracts price using extract_price"""
```
## 🧪 Evaluation Framework
Tester Class
A full evaluation harness that:
- Runs predictions on test data
- Computes:
  - Absolute Error
  - Log-Squared Error (SLE)
  - Color classification (green/orange/red) based on error
  - Displays output and generates a scatter plot of predictions
```
Tester.test(model_predict, test)
```
## 📊 Evaluation Metrics
- Average Error – Mean of absolute differences
- RMSLE – Root Mean Squared Logarithmic Error
- Hits – % of predictions within good range (green)
- Color-coded CLI output – Easy spotting of large misses

## 🎯 Summary
This notebook:
- Uses a quantized LLaMA-3.2 model to predict prices from text descriptions
- Evaluates performance against a labeled dataset from Hugging Face
- Includes a visual and statistical evaluation suite
- Can be adapted for price estimation in e-commerce, scraping, or valuation use cases

In [1]:
# ------------------------------------ Installations ----------------------------------
# pip installs - ignore the error message!

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ------------------------------------ Imports ----------------------------------
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
import matplotlib.pyplot as plt

In [32]:
# ------------------------------------ Tokenizers ----------------------------------

LLAMA_3_1 = "meta-llama/Meta-Llama-3.1-8B"
LLAMA_3_2 = "meta-llama/Llama-3.2-1B"
QWEN_2_5 = "Qwen/Qwen2.5-7B"
GEMMA_2 = "google/gemma-2-9b"
PHI_3 = "microsoft/Phi-3-medium-4k-instruct"

# ------------------------------------ Constants ----------------------------------

BASE_MODEL = LLAMA_3_1
HF_USER = "ed-donner"
DATASET_NAME = f"{HF_USER}/pricer-data"
MAX_SEQUENCE_LENGTH = 182
QUANT_4_BIT = True

# Used for writing to output in color
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

%matplotlib inline

### Log in to HuggingFace

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

In [33]:
# ------------------------------------ Log In to HuggingFace ----------------------------------

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [34]:
def investigate_tokenizer(model_name):
  print("Investigating tokenizer for", model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  for number in [0, 1, 10, 100, 999, 1000]:
    tokens = tokenizer.encode(str(number), add_special_tokens=False)
    print(f"The tokens for {number}: {tokens}")

In [35]:
# Now we will try this with each model: LLAMA_3_1, QWEN_2_5, GEMMA_2, PHI_3

investigate_tokenizer(PHI_3)
investigate_tokenizer(QWEN_2_5)
investigate_tokenizer(GEMMA_2)
investigate_tokenizer(LLAMA_3_1)
investigate_tokenizer(LLAMA_3_2)

Investigating tokenizer for microsoft/Phi-3-medium-4k-instruct
The tokens for 0: [29871, 29900]
The tokens for 1: [29871, 29896]
The tokens for 10: [29871, 29896, 29900]
The tokens for 100: [29871, 29896, 29900, 29900]
The tokens for 999: [29871, 29929, 29929, 29929]
The tokens for 1000: [29871, 29896, 29900, 29900, 29900]
Investigating tokenizer for Qwen/Qwen2.5-7B
The tokens for 0: [15]
The tokens for 1: [16]
The tokens for 10: [16, 15]
The tokens for 100: [16, 15, 15]
The tokens for 999: [24, 24, 24]
The tokens for 1000: [16, 15, 15, 15]
Investigating tokenizer for google/gemma-2-9b
The tokens for 0: [235276]
The tokens for 1: [235274]
The tokens for 10: [235274, 235276]
The tokens for 100: [235274, 235276, 235276]
The tokens for 999: [235315, 235315, 235315]
The tokens for 1000: [235274, 235276, 235276, 235276]
Investigating tokenizer for meta-llama/Meta-Llama-3.1-8B
The tokens for 0: [15]
The tokens for 1: [16]
The tokens for 10: [605]
The tokens for 100: [1041]
The tokens for 999

## Investigating tokenizer for microsoft/Phi-3-medium-4k-instruct
- The tokens for 0: [29871, 29900]
- The tokens for 1: [29871, 29896]
- The tokens for 10: [29871, 29896, 29900]
- The tokens for 100: [29871, 29896, 29900, 29900]
- The tokens for 999: [29871, 29929, 29929, 29929]
- The tokens for 1000: [29871, 29896, 29900, 29900, 29900]

## Investigating tokenizer for meta-llama/Llama-3.2-1B
- The tokens for 0: [15]
- The tokens for 1: [16]
- The tokens for 10: [605]
- The tokens for 100: [1041]
- The tokens for 999: [5500]
- The tokens for 1000: [1041, 15]

```
def investigate_tokenizer(model_name):
  print("Investigating tokenizer for", model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  for number in [0, 1, 10, 100, 999, 1000]:
    tokens = tokenizer.encode(str(number), add_special_tokens=False)
    print(f"The tokens for {number}: {tokens}")
```
- This function is a diagnostic tool to see how a tokenizer from a given LLM model handles numbers.
## 🧠 What It Does — Step by Step
- Prints the name of the model you're about to inspect.
- tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
- Loads the tokenizer associated with the model (e.g., LLaMA, Qwen, Phi-3, etc.)
- trust_remote_code=True allows custom tokenizer classes from model authors.
- for number in [0, 1, 10, 100, 999, 1000]:
    - tokens = tokenizer.encode(str(number), add_special_tokens=False)
    - print(f"The tokens for {number}: {tokens}")
    - Loops through a set of sample numbers, converts each number to a string
    - Uses the tokenizer to convert it into token IDs
    - Prints the result so you can see how the model breaks that number down

## 🧪 Why Use This?
- Different tokenizers treat numbers very differently:
- Some models tokenize "1000" as a single token (great!)
- Others break it into characters or pieces like ['1', '000'] or even digit-by-digit
- This impacts performance in math, price prediction, scientific data, etc.
- Especially important for domains like LCMS, where values like "0.00123" must be tokenized cleanly

## 🔍 Example Output

For "1000", you might see:
- Model A: The tokens for 1000: [27891] → ✅ 1 token
- Model B: The tokens for 1000: [143, 27, 15] → ❌ broken into sub-tokens

## 📊 Token vs. Sub-token for Numerical Data

| Feature              | Single Token (`"1000"` → `[27891]`) | Sub-token (`"1000"` → `[143, 27, 15]`) |
|----------------------|--------------------------------------|----------------------------------------|
| ✅ Accurate Meaning   | Yes                                  | No — split may distort the number      |
| ⚡ Efficiency         | High — fewer tokens                  | Low — more tokens                      |
| 🧠 Model Understanding | Coherent number                     | Fragmented / less clear                |
| 📈 Prediction Quality | Better for regression & reasoning    | Can hurt numeric tasks



# Load our data

We uploaded it to Hugging Face, so it's easy to retrieve it now

In [36]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [37]:
test[0]

{'text': "How much does this cost to the nearest dollar?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K\n\nPrice is $",
 'price': 374.41}

## test[0] returns:
{'text': "How much does this cost to the nearest dollar?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K\n\nPrice is $",
 **'price': 374.41**}

---

We just need our model to predict:  
**'price': 374.41**

# Prepare our Base Llama Model for evaluation

Load our base model with 4 bit quantization and try out 1 example

In [38]:
## pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

### 🧮 Quantization Configuration (4-bit vs 8-bit)

This block sets up the quantization settings using `BitsAndBytesConfig` from the `bitsandbytes` library. It enables loading large language models efficiently by reducing precision and memory usage.

```python
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,                      # Enable 4-bit loading
        bnb_4bit_use_double_quant=True,         # Use double quantization (better compression)
        bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation
        bnb_4bit_quant_type="nf4"               # Use "nf4" quantization (normal float 4)
    )
else:
    quant_config = BitsAndBytesConfig(  
        load_in_8bit=True,                      # Enable 8-bit loading
        bnb_8bit_compute_dtype=torch.bfloat16   # Use bfloat16 for math
    )
```
### 🔹 Key Points:
- 4-bit quantization (NF4) reduces memory usage significantly.
- 8-bit quantization is slightly more accurate but requires more memory.
- Both options allow loading large models (like LLaMA or Mistral) on GPUs with limited VRAM (e.g., 8–16GB).
- bfloat16 is used for computation for improved performance on modern GPUs.
- Use this configuration when loading models with:
```
AutoModelForCausalLM.from_pretrained(..., quantization_config=quant_config)
```

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:.1f} GB")

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

In [ ]:
def extract_price(s):
    """
    Extracts the first numeric price value from a string that contains the phrase "Price is $".

    Parameters:
        s (str): Input string potentially containing a price in the format "Price is $<amount>".

    Returns:
        float: The extracted price as a float. Returns 0 if no price is found.

    Example:
        >>> extract_price("Special deal! Price is $1,299.99 only today!")
        1299.99

    Notes:
        - Only extracts the first price following the exact phrase "Price is $"
        - Removes commas and dollar signs before extracting the number
        - Returns 0 if the pattern is not found or if no number follows
    """
    if "Price is $" in s:
        contents = s.split("Price is $")[1]
        contents = contents.replace(',', '').replace('$', '')
        match = re.search(r"[-+]?\d*\.\d+|\d+", contents)
        return float(match.group()) if match else 0
    return 0

In [ ]:
extract_price("Price is $999 blah blah so cheap")

In [ ]:
def model_predict(prompt):
    """
    Generates a response from the language model based on the given prompt
    and extracts the first numeric price from the output.

    Parameters:
        prompt (str): Input prompt to feed into the model, typically containing
                      product or descriptive text that implies a price.

    Returns:
        float: The first numeric price extracted from the model's generated response.
               Returns 0 if no price is found.

    Process:
        - Sets the random seed for reproducibility.
        - Encodes the prompt using the tokenizer.
        - Generates a short response (max 4 new tokens) using the base model.
        - Decodes the output tokens to text.
        - Extracts and returns the first price found in the response using `extract_price()`.

    Notes:
        - Assumes the model and tokenizer are already loaded and moved to GPU.
        - Output length is limited to 4 new tokens for focused prediction.
        - This function is useful in evaluation or prediction tasks involving
          price inference from text.
    """
    set_seed(42)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    attention_mask = torch.ones(inputs.shape, device="cuda")
    outputs = base_model.generate(
        inputs,
        max_new_tokens=4,
        attention_mask=attention_mask,
        num_return_sequences=1
    )
    response = tokenizer.decode(outputs[0])
    return extract_price(response)


In [ ]:
model_predict(test[0]['text'])

## Returns:  
1999.0

# Evaluation!

Trying out our base Llama 3.2 model against the Test dataset

In [ ]:
class Tester:

    def __init__(self, predictor, data, title=None, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint["text"])
        truth = datapoint["price"]
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint["text"].split("\n\n")[1][:20] + "..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function, data):
        cls(function, data).run()

In [ ]:
Tester.test(model_predict, test)